In [1]:
# =============================================================================
# bin/run_model.py
# @author: Brian Kyanjo
# @date: 2024-09-24
# @description: This script runs the flowline model using the EnKF data assimilation scheme.
# =============================================================================

In [2]:
# import libraries ========
import os
import sys
import jax
import warnings
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
from scipy.optimize import root
from scipy.stats import norm, multivariate_normal

warnings.filterwarnings("ignore")
jax.config.update("jax_enable_x64", True) # Set the precision in JAX to use float64

In [3]:
# Add the parent and `src` directories to sys.path if needed
for path in [os.path.abspath('..'), os.path.abspath('../src')]:
    if path not in sys.path:
        sys.path.insert(0, path)

# Import necessary modules
from config.config_loader import ParamsLoader
from EnKF.python_enkf.enkf_class_python import EnsembleKalmanFilter
from utils.utils import UtilsFunctions

config_path = os.path.join(os.getcwd(), '../config', 'params.yaml') 

In [4]:
# Initial setup from the config file and initial guess ---------------
params = ParamsLoader(config_path).get_params()
grid = params['grid']

# Create an instance of UtilsFunctions with params
utils_functions = UtilsFunctions(params)

# Initial guess and steady state ====================
xg = 300e3 / params["xscale"]
hf = (-utils_functions.bed(xg * params["xscale"]) / params["hscale"]) / (1 - params["lambda"])
h  = 1 - (1 - hf) * grid["sigma"]
u  = 1.0 * (grid["sigma_elem"] ** (1 / 3)) + 1e-3
huxg_old = np.concatenate((h, u, [xg]))

huxg_old

array([1.00266118e+00, 1.00535648e+00, 1.00805177e+00, 1.01074707e+00,
       1.01344237e+00, 1.01613767e+00, 1.01883296e+00, 1.02152826e+00,
       1.02422356e+00, 1.02691886e+00, 1.02961415e+00, 1.03230945e+00,
       1.03500475e+00, 1.03770005e+00, 1.04039534e+00, 1.04309064e+00,
       1.04578594e+00, 1.04848124e+00, 1.05117653e+00, 1.05387183e+00,
       1.05656713e+00, 1.05926242e+00, 1.06195772e+00, 1.06465302e+00,
       1.06734832e+00, 1.07004361e+00, 1.07273891e+00, 1.07543421e+00,
       1.07812951e+00, 1.08082480e+00, 1.08352010e+00, 1.08621540e+00,
       1.08891070e+00, 1.09160599e+00, 1.09430129e+00, 1.09699659e+00,
       1.09969189e+00, 1.10238718e+00, 1.10508248e+00, 1.10777778e+00,
       1.10811111e+00, 1.10844444e+00, 1.10877778e+00, 1.10911111e+00,
       1.10944444e+00, 1.10977778e+00, 1.11011111e+00, 1.11044444e+00,
       1.11077778e+00, 1.11111111e+00, 1.00000000e-03, 3.31435469e-01,
       3.93218632e-01, 4.39983204e-01, 4.78470029e-01, 5.11587765e-01,
      

In [6]:
params["xscale"]

168252.74158571838